# Equations used in carbonate chemistry model


    A notebook that annotates eqns we use in the cc model.
    Compares between SKOG and SOG, also notes on questions I have (I use the QQ tag for easy searching). 

    To fill out for DIC, TA, OXY

            1. code location (SKOG & SOG)
            2. relevant literature references
                TA/DIC - MMetal16
                O2 - need to discuss, ref in SOG code not given
            3. outstanding questions


# TA

    
    change TA =  (1 + redfield(P_N))*(uptake NO3 for all phyto) + (redfield(P_N) - 1 ) * (sum uptake NH4 - remineralization) - 2* remineralization no3

<img src="./TA.png"> 

###### SMELT_c4 - p4zcar

                zz_loc_remin_NH4 = zz_remin_D_DON*trb(ji,jj,jk,jpdon) + zz_remin_D_PON*trb(ji,jj,jk,jppon)
                zz_loc_remin_NO3 = zz_remin_NH*trb(ji,jj,jk,jpnh4)*trb(ji,jj,jk,jpnh4)

                tra(ji,jj,jk,jpta) = (zz_redfield_p_n + 1.0) * &
                (zz_uptake_NO_diat(ji,jj,jk)*trb(ji,jj,jk,jpdia)+ &
                 zz_uptake_NO_myri(ji,jj,jk)*trb(ji,jj,jk,jpmes)+&
                 zz_uptake_NO_nano(ji,jj,jk)*trb(ji,jj,jk,jpphy))+ &
                 (zz_redfield_p_n - 1.0) * &
                (zz_uptake_NH_diat(ji,jj,jk)*trb(ji,jj,jk,jpdia)+ &
                 zz_uptake_NH_myri(ji,jj,jk)*trb(ji,jj,jk,jpmes)+&
                 zz_uptake_NH_nano(ji,jj,jk)*trb(ji,jj,jk,jpphy) &
                 - zz_loc_remin_NH4)  - 2 * zz_loc_remin_NO3

###### sog - NPZD
    ! Alkalinity
    bPZ = (PZ_bins%Alk - 1) * M + 1
    ePZ = PZ_bins%Alk * M
    where (Alk > 0.)
       dPZdt(bPZ:ePZ) = ((1 + Redfield_NP) * uptake%NO + (1 - Redfield_NP) &
            * (uptake%NH - remin_NH)) * (1 / Redfield_NP) - 2 * NH_oxid
    endwhere

###### NB/? :
    -the term (1 / Redfield_NP) in the SOG alkalinity is not there in the published equations or in the SMELT_c4 alkalinity - SOLVED
    -sog only has one phyto class, right? for uptake%NH - SOLVED
    -is it a problem that we are calculating zz_loc_remin_NH4 twice in SKOG? 

# DIC

    change DIC = remin - sum(uptake No3, NH4, PC for each phyto class) * redfield_C:N

<img src="./DIC.png"> 

###### SMELT_c4 - p4zcar.f90

                zz_loc_remin_NH4 = zz_remin_D_DON*trb(ji,jj,jk,jpdon) + zz_remin_D_PON*trb(ji,jj,jk,jppon)

               tra(ji,jj,jk,jpdic) = ( zz_loc_remin_NH4 - &
               (zz_uptake_NO_diat(ji,jj,jk)+zz_uptake_NH_diat(ji,jj,jk)+zz_uptake_PC_diat(ji,jj,jk))*trb(ji,jj,jk,jpdia) + &
               (zz_uptake_NO_nano(ji,jj,jk)+zz_uptake_NH_nano(ji,jj,jk)+zz_uptake_PC_nano(ji,jj,jk))*trb(ji,jj,jk,jpphy) + &
               (zz_uptake_NO_myri(ji,jj,jk)+zz_uptake_NH_myri(ji,jj,jk)+zz_uptake_PC_myri(ji,jj,jk))*trb(ji,jj,jk,jpmes) &
                                                        ) * zz_redfield_c_n
                                                        
###### In SOG -  NPZD.f90

    ! Dissolved inorganic carbon
    bPZ = (PZ_bins%DIC - 1) * M + 1
    ePZ = PZ_bins%DIC * M
    where (DIC > 0.)
       dPZdt(bPZ:ePZ) = (remin_NH - uptake%NO - uptake%NH - uptake%PC) &
                  * Redfield_C
    endwhere
    
    
###### NB/? :
    -sog only has one phyto class, right? for uptake%NH
    -is it a problem that we are calculating zz_loc_remin_NH4 twice in the SMELT_c4? 
    

# oxygen 

Equations - TD need to write into LaTeX/ find where they are

### SOG - NPZD.f90
 
        ! Dissolved oxygen
    bPZ = (PZ_bins%Oxy - 1) * M + 1
    ePZ = PZ_bins%Oxy * M
    where (Oxy > 0.)
       dPZdt(bPZ:ePZ) = (uptake%NO + uptake%NH - remin_NH - NH_oxid) &
                  * Redfield_O
    endwhere
    
    temp_Q10 = dexp (0.07 * (KtoC(Temp(1:M)) - 20.d0))
    NH_oxid = remin%NH * NH**2 * temp_Q10
    

### SKOG - p4zcar.f90


           SUBROUTINE p4z_oxy( kt, knt, zz_uptake_NO_diat, zz_uptake_NO_nano, zz_uptake_NO_myri, &
                                zz_uptake_NH_diat, zz_uptake_NH_nano, zz_uptake_NH_myri)
              !!---------------------------------------------------------------------
              !!                     ***  ROUTINE p4z_oxy  ***
              !!
              !! ** Purpose :   Compute change in O2
              !!
              !! ** Method  : - ???
              !!---------------------------------------------------------------------
              INTEGER  ::   ji, jj, jk
              INTEGER, INTENT(in) ::   kt, knt ! ocean time step
              REAL(wp), INTENT(in)   :: zz_uptake_NO_diat(:,:,:), zz_uptake_NO_nano(:,:,:), zz_uptake_NO_myri(:,:,:), &
                                        zz_uptake_NH_diat(:,:,:), zz_uptake_NH_nano(:,:,:), zz_uptake_NH_myri(:,:,:)

              REAL(wp) :: zz_loc_remin_NH4
              REAL(wp) :: zz_loc_oxy_NH4
              REAL(wp) :: zz_loc_temp_Q10
              IF( nn_timing == 1 )  CALL timing_start('p4z_oxy')

              DO jk = 1, jpkm1
                 DO jj = 1, jpj
                    DO ji = 1, jpi


                        zz_loc_temp_Q10 = dexp(0.07 * ( (tsn(ji,jj,jk,jp_tem)) - 20 ))
                        zz_loc_remin_NH4 = zz_remin_D_DON*trb(ji,jj,jk,jpdon) + zz_remin_D_PON*trb(ji,jj,jk,jppon)
                        zz_loc_oxy_NH4 = zz_loc_remin_NH4 * trb(ji,jj,jk,jpnh4) ** 2 * zz_loc_temp_Q10

                       tra(ji,jj,jk,jpo2) = ( &
                       (zz_uptake_NO_diat(ji,jj,jk)+zz_uptake_NH_diat(ji,jj,jk))*trb(ji,jj,jk,jpdia) + &
                       (zz_uptake_NO_nano(ji,jj,jk)+zz_uptake_NH_nano(ji,jj,jk))*trb(ji,jj,jk,jpphy) + &
                       (zz_uptake_NO_myri(ji,jj,jk)+zz_uptake_NH_myri(ji,jj,jk))*trb(ji,jj,jk,jpmes) &
                        - zz_loc_remin_NH4 ) * zz_redfield_o_n
                    END DO
                 END DO
              END DO
              IF( nn_timing == 1 )  CALL timing_stop('p4z_oxy')


           END SUBROUTINE p4z_oxy